# install library

In [1]:
import os

if os.environ.get("KAGGLE_KERNEL_RUN_TYPE") is None:
    ON_KAGGLE = False
else:
    ON_KAGGLE = True
if not ON_KAGGLE:
    import shutil
    from requests import get

    from google.colab import drive, files
    # mount Google Drive
    drive.mount("/content/drive")
else:
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
if not ON_KAGGLE: 
    !pip uninstall -y kaggle
    !pip install kaggle
    !mkdir /root/.kaggle
    !cp /content/drive/MyDrive/kaggla-za/kaggle.json /root/.kaggle
    !kaggle datasets download -d  kozodoi/timm-pytorch-image-models
    !mkdir -p ../input/timm-pytorch-image-models
    !unzip -n -qq timm-pytorch-image-models.zip -d ../input/timm-pytorch-image-models

    #一回のみ
    # !kaggle datasets download -d kalfirst/animal-imbalance-classification-round2
    # !mkdir -p ../content/drive/MyDrive/kaggla-za/E002Round3/Week2/AnimalImbalance/data/animal-imbalance-classification-round2
    # !unzip -n -qq animal-imbalance-classification-round2.zip -d ../content/drive/MyDrive/kaggla-za/E002Round3/Week2/AnimalImbalance/data/animal-imbalance-classification-round2

    !kaggle datasets download -d kalfirst/animal-imbalance-classification-round2
    !mkdir -p ../input/animal-imbalance-classification-round2
    !unzip -n -qq animal-imbalance-classification-round2.zip -d ../input/animal-imbalance-classification-round2


Found existing installation: kaggle 1.5.12
Uninstalling kaggle-1.5.12:
  Successfully uninstalled kaggle-1.5.12
  Using cached kaggle-1.5.12-py3-none-any.whl
mkdir: cannot create directory ‘/root/.kaggle’: File exists
timm-pytorch-image-models.zip: Skipping, found more recently modified local copy (use --force to force download)
animal-imbalance-classification-round2.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import sys
sys.path.append("../input/timm-pytorch-image-models/pytorch-image-models-master")

if not ON_KAGGLE:
    !pip install albumentations==0.4.6
import albumentations as A
from albumentations.pytorch import ToTensorV2

import gc
import math
import os
import pickle
import random
import re
import time
import warnings
from contextlib import contextmanager

from pathlib import Path
from PIL import Image

#import albumentations as A
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import torch
import torch.nn as nn
import torch.nn.functional as F


#from albumentations.pytorch import ToTensorV2
from scipy.special import softmax
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from torch.optim import SGD, Adam, AdamW, RAdam
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts, ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm

import timm
warnings.filterwarnings('ignore')

# Config


In [4]:
class Config:
    Version = "exp35" # model save時のversion
    #MODEL_NAME = "resnet34" # モデル名
    #MODEL_NAME = "efficientnet_b0" # モデル名
    #MODEL_NAME = "efficientnet_b1" # モデル名
    #MODEL_NAME = "efficientnet_b2" # モデル名
    #MODEL_NAME = "efficientnet_b3" # モデル名
    #MODEL_NAME = "efficientnet_b4" # モデル名
    #MODEL_NAME = "efficientnet_b5" # モデル名
    #MODEL_NAME = "efficientnet_b6" # モデル名
    #MODEL_NAME = "efficientnet_b7" # モデル名
    #MODEL_NAME = "tf_efficientnet_b0_ns" # モデル名
    MODEL_NAME = "tf_efficientnet_b1_ns" # モデル名
    #MODEL_NAME = "resnet50" # モデル名
    # MODEL_NAME = "swin_large_patch4_window7_224"
    # MODEL_NAME = "swin_base_patch4_window7_224"
    #MODEL_NAME = "swin_small_patch4_window7_224"
    # MODEL_NAME = "swin_large_patch4_window12_384"
    # MODEL_NAME = "swin_base_patch4_window12_384"
    # MODEL_NAME = "vit_base_patch16_224"


    if ON_KAGGLE:
        # if MODEL_NAME == 'resnet34':
        #     model_name = '../input/pt-longformer-base' # https://www.kaggle.com/kishalmandal/pt-longformer-base
        # elif MODEL_NAME == 'roberta-base':
        #     model_name = '../input/roberta-base' #https://www.kaggle.com/abhishek/roberta-base
        # elif modeMODEL_NAMEl_savename == 'roberta-large':
        #     model_name = '../input/robertalarge' # https://www.kaggle.com/marshal02/robertalarge
        # elif MODEL_NAME == 'longformer-large':
        #     model_name = '../input/longformerlarge4096' # https://www.kaggle.com/hengzheng/longformerlarge4096 
        #base_dir = '/content/drive/MyDrive/petfinder'
        #data_dir = '../input/digit-recognizer/' # = DATA_PATH
        data_dir = '../input/animal-imbalance-classification-round2' # = DATA_PATH
        pre_data_dir = './preprocessed/'
        model_dir = '.'
        output_dir = '.'   
    else:
        # customize for my own Google Colab Environment
        # if model_savename == 'longformer':
        #     model_name = 'allenai/longformer-base-4096'
        # elif model_savename == 'roberta-base':
        #     model_name = 'roberta-base'
        base_dir = '/content/drive/MyDrive/kaggla-za/E002Round3/Week2/AnimalImbalance'
        data_dir = '../input/animal-imbalance-classification-round2' #os.path.join(base_dir, '../input/animal-imbalance-classification-round2')
        #pre_data_dir = os.path.join(base_dir, 'data/preprocessed')
        model_dir = os.path.join(base_dir, f'model/{Version}')
        output_dir = os.path.join(base_dir, f'output/{Version}')    

    is_debug = False
    if MODEL_NAME == 'resnet34':
        size = 224 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    elif MODEL_NAME == 'roberta-base':
        size = 224 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    elif MODEL_NAME == 'resnet50':
        size = 64 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    elif MODEL_NAME == 'swin_large_patch4_window7_224':
        size = 224 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    elif MODEL_NAME == 'swin_base_patch4_window7_224':
        size = 224 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    elif MODEL_NAME == 'swin_small_patch4_window7_224':
        size = 224 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    elif MODEL_NAME == 'swin_large_patch4_window12_384':
        size = 384 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    elif MODEL_NAME == 'swin_base_patch4_window12_384':
        size = 384 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    elif MODEL_NAME == 'efficientnet_b0':
        size = 224 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    elif MODEL_NAME == 'efficientnet_b1':
        size = 224 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    elif MODEL_NAME == 'efficientnet_b2':
        size = 224 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    elif MODEL_NAME == 'efficientnet_b3':
        size = 224 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    elif MODEL_NAME == 'efficientnet_b4':
        size = 224 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    elif MODEL_NAME == 'efficientnet_b5':
        size = 224 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    elif MODEL_NAME == 'efficientnet_b6':
        size = 224 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    elif MODEL_NAME == 'efficientnet_b7':
        size = 224 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    elif MODEL_NAME == 'tf_efficientnet_b0_ns':
        size = 224 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    elif MODEL_NAME == 'tf_efficientnet_b1_ns':
        size = 224 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    elif MODEL_NAME == 'vit_base_patch16_224':
        size = 224 # リサイズした後の画像のサイズ
        model_lr = 1e-5
    seed = 3037 # seed値
    fold = 3
    FOLDS = [0,1,2]
    stratified = True
    num_workers = 4  #並列実行する数 was 0
    batch_size =  64 #バッチサイズ was 64 >100
    epochs = 50 #エポック数 10?
    #size = 224 # リサイズした後の画像のサイズ
    #model_lr = 1e-5 # 学習率
    T_max = 10 # 最大イテレーション数 was 10
    min_lr = 1e-3 # 学習率の最小値 was 1e-5
    weight_decay = 1e-6 # 学習減衰値
    max_grad_norm = 1000 # 勾配の最大ノルム
    gradient_accumulation_steps = 1
    print_freq = 1000 # 学習結果を表示する頻度
    probability = 0.6
    class_num = 10 
    norms = (0.5, 0.5)
    gamma = 2
    device = "cuda" if torch.cuda.is_available() else "cpu" # CPU of GPU
    debug = False


In [5]:
if not ON_KAGGLE:
    if not os.path.exists(Config.model_dir):
        !mkdir $Config.model_dir
    if not os.path.exists(Config.output_dir):
        !mkdir $Config.output_dir

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

# Utils

In [7]:
def init_logger(log_file='train.log'):
    """Output Log."""
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger


def seed_torch(seed=Config.seed): # was seed=0
    """Fixed seed value."""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # added trueにすると再現性がなくなる


LOGGER = init_logger()
seed_torch(seed=Config.seed)

In [8]:
def asMinutes(s):
    """Convert Seconds to Minutes."""
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    """Accessing and Converting Time Data."""
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

class AverageMeter(object):
    """Computes and stores the average and current value."""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

# Fold

In [9]:
def make_fold(df, stratified=Config.stratified):
    """Add folds column."""
    df["folds"] = -1
    if stratified:
        kf = StratifiedKFold(n_splits=Config.fold, shuffle=True, random_state=Config.seed)
        for f, (t_, v_) in enumerate(kf.split(X=df, y=df.label.values)):
            df.loc[v_, 'folds'] = f
        df["folds"] = df["folds"].astype(int)

    else:
        kf = KFold(n_splits=Config.fold, shuffle=True, random_state=Config.seed)
        for f, (t_, v_) in enumerate(kf.split(df)):
            df.loc[v_, 'folds'] = f
    return df


# def get_train_file_path(image_id):
#     """Get test img path."""
#     return "../input/animal-imbalance-classification-round2/train_images/{}".format(
#         image_id
#     )

# Dataset

In [10]:
class TrainDataset(Dataset):
    """Dataset used for training."""
    def __init__(self, df, transform=None):
        self.df = df
        self.image_paths = df['image_id'].values
        self.labels = df['label'].values
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        #修正部分
        path = self.image_paths[idx]
        image = np.array(Image.open(path))
        # image = self.df.loc[idx, "pixel0": "pixel783"].values.astype(np.uint8).reshape(28, 28) # csv to image
        # image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB) # GRAY to RGB

        if self.transform:
            augmented = self.transform(image=image) # augmentation
            image = augmented['image']

        label = self.labels[idx] # 正解ラベル
        return image, label

In [11]:
class TestDataset(Dataset):
    """Dataset used for inference."""
    def __init__(self, df, transform=None):
        self.df = df
        self.image_paths = df['image_id'].values
        self.labels = df['label'].values
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        #修正部分
        path = self.image_paths[idx]
        image = np.array(Image.open(path))
        # image = self.df.loc[idx, "pixel0": "pixel783"].values.astype(np.uint8).reshape(28, 28)
        # image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        return image

# Data Augmentation

In [12]:
def get_transforms(*, data, size, norms):

    if data == 'train':
        return A.Compose([
            A.Resize(Config.size, Config.size), # リサイズ
            A.Rotate(limit=10, border_mode=cv2.BORDER_REPLICATE, p=0.5),
            A.Cutout(num_holes=8, max_h_size=2, max_w_size=2, fill_value=0, p=0.5),
            A.Cutout(num_holes=8, max_h_size=1, max_w_size=1, fill_value=1, p=0.5),
            A.VerticalFlip(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.Normalize(mean=[norms[0], norms[0], norms[0]], std=[norms[1], norms[1], norms[1]], max_pixel_value=255.0, p=1.0), # 正規化
            ToTensorV2(), # Tensor型に変換
        ])

    elif data == 'valid':
        return A.Compose([
            A.Resize(Config.size, Config.size),
            A.Normalize(mean=[norms[0], norms[0], norms[0]], std=[norms[1], norms[1], norms[1]], max_pixel_value=255.0, p=1.0),
            ToTensorV2(),
        ])

# Model

In [13]:
# use timm model no pretrained
class BaseModel(nn.Module):
    def __init__(
        self, model_name=Config.MODEL_NAME, n_class=10, pretrained=False
    ):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained) # model load(pretrained=False: 事前学習なし)
        n_features = self.model.num_features#fc.in_features
        self.model.classifier = nn.Linear(n_features, n_class) # 出力層の次元をクラス数に変更

    def forward(self, x):
        output = self.model(x)
        return output

# Loss

In [ ]:
class FocalLoss(nn.Module):

    def __init__(self, gamma=Config.gamma, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps
        self.ce = torch.nn.CrossEntropyLoss(reduction="none")

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

# train & valid function

In [14]:
def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()

    model.train() # 学習モード
    start = end = time.time()
    global_step = 0

    for step, (images, labels) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(Config.device) # 画像をcpuからgpuヘ
        labels = labels.to(Config.device) # 正解ラベルをcpuからgpuヘ
        batch_size = labels.size(0) 
        y_preds = model(images) # 予測ラベル
        loss = criterion(y_preds, labels) # lossの計算

        losses.update(loss.item(), batch_size) 
        if Config.gradient_accumulation_steps > 1:
            loss = loss / Config.gradient_accumulation_steps
        else:
            loss.backward()
            
        del loss
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), Config.max_grad_norm)

        if (step + 1) % Config.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        # measure elapsed time
        

        batch_time.update(time.time() - end)
        end = time.time()
        if step % Config.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(
                   epoch+1, step, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(train_loader)),
                   grad_norm=grad_norm,
                   ))

        del y_preds, images, labels, batch_size
    del batch_time, data_time, scores, model
    gc.collect()
    torch.cuda.empty_cache()
    
    return losses.avg

In [15]:
def valid_fn(valid_loader, model, criterion):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()

    # switch to evaluation mode
    model.eval()
    preds = []
    confs = []
    start = end = time.time()
    for step, (images, labels) in enumerate(valid_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(Config.device)
        labels = labels.to(Config.device)
        batch_size = labels.size(0)
        with torch.no_grad():
        #with torch.inference_mode():
            y_preds = model(images)
            loss = criterion(y_preds, labels)
            losses.update(loss.item(), batch_size)
            # record accuracy
            preds.append(y_preds.softmax(1).to("cpu").numpy().argmax(1))

            if Config.gradient_accumulation_steps > 1:
                loss = loss / Config.gradient_accumulation_steps
            
            del loss
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
            if step % Config.print_freq == 0 or step == (len(valid_loader)-1):
                print('EVAL: [{0}/{1}] '
                    'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                    'Elapsed {remain:s} '
                    'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                    .format(
                    step, len(valid_loader), batch_time=batch_time,
                    data_time=data_time, loss=losses,
                    remain=timeSince(start, float(step+1)/len(valid_loader)),
                    ))
        del y_preds, images, labels, batch_size
    del batch_time, data_time, scores, model
    gc.collect()
    torch.cuda.empty_cache()

    preds = np.concatenate(preds)

    return losses.avg, preds

# Dataの確認

In [16]:
#data_dir = "../input/digit-recognizer" # ディレクトリパス = data_dir
if ON_KAGGLE:
    train = make_fold(pd.read_csv(Config.data_dir + "/train.csv")) # csvの読み込み
    train['image_id'] = Config.data_dir + '/train_images/' + train['image_id']
else:
    train = make_fold(pd.read_csv(f'{Config.data_dir}/train.csv'))
    train['image_id'] = Config.data_dir + '/train_images/' + train['image_id']

train.head()

,image_id,label,folds
0,../input/animal-imbalance-classification-round...,0,0
1,../input/animal-imbalance-classification-round...,0,2
2,../input/animal-imbalance-classification-round...,0,0
3,../input/animal-imbalance-classification-round...,0,0
4,../input/animal-imbalance-classification-round...,0,2


In [17]:
#train['image_id'][0]

In [18]:
import plotly.express as px

target = train.label.value_counts()
#class_num = len(target)

fig = px.pie(target,
             values='label',
             names=target.index,
             hole=.4, 
             width=500, height=500)
fig.update_traces(textinfo='value+label', pull=0.01)
fig.show()

In [19]:
fig = px.bar(target.sort_index().index, target.sort_index(), width=1000, height=400)
fig.show()

# 前処理

### ホールドアウト

In [20]:
# f0, f1 = train_test_split(train, test_size=0.2) # 学習用と評価用に分ける
# train["folds"] = -1
# train.loc[f0.index, "folds"] = 1
# train.loc[f1.index, "folds"] = 0
# train

#### 平均と標準偏差を計算

In [21]:
#修正部分
# images = train.loc[:, "pixel0": "pixel783"].values.astype(np.uint8).reshape(-1, 28, 28)
# norms = (np.mean(images[train.folds!=0] / 255), np.std(images[train.folds!=0] / 255)) # 平均と標準偏差を計算
# print("平均と標準偏差: ", norms)

#### 画像サイズの確認と数例を表示

In [22]:
# norms = (0.5, 0.5)
# train_dataset = TrainDataset(train,transform=get_transforms(data='train', size=Config.size, norms=Config.norms)) # 学習用のdatasetを作成
# train_loader = DataLoader(train_dataset, batch_size=Config.batch_size, shuffle=True, num_workers=Config.num_workers, pin_memory=True, drop_last=True)# 学習用のdatasetsのbatchを作成
# image_iter = iter(train_loader)
# images, labels = image_iter.next()

In [23]:
# print("height: ", images.shape[1])
# print("weight: ", images.shape[2])

# for n in range(3):
#     fig, axes = plt.subplots(1, 10, figsize=(20, 30))
#     [axes[i].imshow(images[idx]) for i, idx in enumerate(range(n*10, (n+1)*10))]

#     plt.show()

# 学習の際必要になる知識

## 評価指標(Accuracy), Confusion matrix
#### 詳細については、来週説明

## ハイパラメータの用語簡易説明など

> #### * 学習率 : 学習一回当たりの更新量のようなものです。 今回のデフォルトは1e-5です。
> #### * スケジューラー : epoch数に応じて、学習率を変化させるものです。 今回のデフォルトはCosineAnnealingLRです。
> #### * 損失関数: 正解ラベルと予測ラベルの誤差の大きさを計算する関数。 
> #### * 最適化関数: 効率よく損失を減らすための関数。 
> #### * timm : 画像のコンペでpytorchを使う場合にほぼ確実に使用するライブラリですので、見ておきましょう。[github](https://github.com/rwightman/pytorch-image-models)

# 軽めのモデル作成

### モデルの読み込み

In [24]:
# model = BaseModel(Config.MODEL_NAME, Config.class_num)
# model

### 最適化関数

In [25]:
# Adam は勾配を移動平均と学習率の調整からなります (MomentumとRMSProp)
#optimizer = Adam(model.parameters(), lr=Config.model_lr, weight_decay=Config.weight_decay, amsgrad=False)
#optimizer = AdamW(model.parameters(), lr=Config.model_lr, weight_decay=Config.weight_decay, amsgrad=False)
# optimizer = RAdam(model.parameters(), lr=Config.model_lr, weight_decay=Config.weight_decay)


### スケジューラ

In [26]:
# scheduler = CosineAnnealingLR(optimizer, T_max=Config.T_max, eta_min=0.001)

# log_s = []
# for epoch in range(0, 100):
#     log_s.append(scheduler.get_last_lr()[0])
#     scheduler.step()

# plt.figure(figsize=(10, 5))
# plt.plot(np.arange(100), log_s)
# plt.show()

# 学習

In [27]:
def train_loop(train, fold, class_num=None, norms=(0.5, 0.5)):
    
    train_index = train[train['folds'] != fold].index # 学習用のindex
    valid_index = train[train['folds'] == fold].index # 評価用のindex

    train_dataset = TrainDataset(train.loc[train_index].reset_index(drop=True), 
                                 transform=get_transforms(data='train', size=Config.size, norms=norms)) # 学習用のdatasetを作成
    valid_dataset = TrainDataset(train.loc[valid_index].reset_index(drop=True), 
                                 transform=get_transforms(data='valid', size=Config.size, norms=norms)) # 評価用のdatasetを作成

    train_loader = DataLoader(train_dataset, batch_size=Config.batch_size, shuffle=True, 
                              num_workers=Config.num_workers, pin_memory=True, drop_last=True)# 学習用のdatasetsのbatchを作成
    valid_loader = DataLoader(valid_dataset, batch_size=Config.batch_size, shuffle=False, 
                              num_workers=Config.num_workers, pin_memory=True, drop_last=False) # 評価用のdatasetsのbatchを作成
    
    
    model = BaseModel(Config.MODEL_NAME, class_num).to(Config.device)

    #optimizer = Adam(model.parameters(), lr=Config.model_lr, weight_decay=Config.weight_decay, amsgrad=False)
    #optimizer = AdamW(model.parameters(), lr=Config.model_lr, weight_decay=Config.weight_decay, amsgrad=False)
    optimizer = RAdam(model.parameters(), lr=Config.model_lr, weight_decay=Config.weight_decay)

    scheduler = CosineAnnealingLR(optimizer, T_max=Config.T_max, eta_min=Config.min_lr, last_epoch=-1)

    criterion = FocalLoss() #nn.CrossEntropyLoss() # loss function

    best_score = 0.
    best_loss = np.inf
    
    for epoch in range(Config.epochs):
        start_time = time.time()
        avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler) # 学習
        avg_val_loss, predict = valid_fn(valid_loader, model, criterion) # 評価用の推論
        valid_labels = train.loc[valid_index, "label"].values # 予測ラベル
        
        scheduler.step() 
        score = accuracy_score(valid_labels, predict) # 適合率を計算
        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Accuracy: {score}')

        if score > best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            model_filename = f'{Config.model_dir}/{Config.MODEL_NAME}_fold{fold}_best_{Config.Version}.pth'
            torch.save(model.state_dict(), model_filename) # モデルの保存
            #torch.save(model.state_dict(), "./" + f'{Config.MODEL_NAME}_fold{fold}_best_{Config.Version}.pth') # モデルの保存
            print(f'{model_filename} saved')

            valid_folds_predict = predict
            
        del avg_loss, avg_val_loss, valid_labels, predict, score
        gc.collect()
        torch.cuda.empty_cache()

    return valid_folds_predict

In [28]:
oof_df = pd.DataFrame()
preds = []
for fold in range(Config.fold):
    if fold not in Config.FOLDS:
        continue

    print(f"========== fold: {fold} ==========")
    if Config.debug:
        train = train.iloc[:2000]
    pred = train_loop(train, fold, Config.class_num)
    preds.append([pred])
    LOGGER.info(f"========== fold: {fold} result ==========")

========== fold: 0 ==========
Epoch: [1][0/380] Data 1.420 (1.420) Elapsed 0m 2s (remain 15m 56s) Loss: 2.3452(2.3452) 
Epoch: [1][379/380] Data 0.000 (0.005) Elapsed 2m 24s (remain 0m 0s) Loss: 2.2598(2.2730) 
EVAL: [0/191] Data 0.552 (0.552) Elapsed 0m 0s (remain 2m 13s) Loss: 2.4097(2.4097) 
EVAL: [190/191] Data 0.000 (0.009) Elapsed 0m 21s (remain 0m 0s) Loss: 2.4275(2.2799) 


Epoch 1 - avg_train_loss: 2.2730  avg_val_loss: 2.2799  time: 167s
Epoch 1 - Accuracy: 0.15221500780800526
Epoch 1 - Save Best Score: 0.1522 Model


/content/drive/MyDrive/kaggla-za/E002Round3/Week2/AnimalImbalance/model/exp35/tf_efficientnet_b1_ns_fold0_best_exp35.pth saved
Epoch: [2][0/380] Data 0.874 (0.874) Elapsed 0m 1s (remain 9m 44s) Loss: 2.2286(2.2286) 
Epoch: [2][379/380] Data 0.000 (0.003) Elapsed 2m 17s (remain 0m 0s) Loss: 2.0213(2.1402) 
EVAL: [0/191] Data 0.524 (0.524) Elapsed 0m 0s (remain 2m 4s) Loss: 2.0581(2.0581) 
EVAL: [190/191] Data 0.000 (0.012) Elapsed 0m 22s (remain 0m 0s) Loss: 3.3894(2.0913) 


Epoch 2 - avg_train_loss: 2.1402  avg_val_loss: 2.0913  time: 161s
Epoch 2 - Accuracy: 0.20859702473904826
Epoch 2 - Save Best Score: 0.2086 Model


/content/drive/MyDrive/kaggla-za/E002Round3/Week2/AnimalImbalance/model/exp35/tf_efficientnet_b1_ns_fold0_best_exp35.pth saved
Epoch: [3][0/380] Data 0.816 (0.816) Elapsed 0m 1s (remain 9m 28s) Loss: 1.9425(1.9425) 
Epoch: [3][379/380] Data 0.000 (0.003) Elapsed 2m 18s (remain 0m 0s) Loss: 1.4775(1.7026) 
EVAL: [0/191] Data 0.760 (0.760) Elapsed 0m 0s (remain 2m 49s) Loss: 1.6433(1.6433) 
EVAL: [190/191] Data 0.000 (0.010) Elapsed 0m 21s (remain 0m 0s) Loss: 4.1341(1.5971) 


...

Epoch 48 - avg_train_loss: 0.1862  avg_val_loss: 0.5445  time: 160s
Epoch 48 - Accuracy: 0.8416899556140063


Epoch: [49][0/380] Data 0.863 (0.863) Elapsed 0m 1s (remain 9m 44s) Loss: 0.1966(0.1966) 
Epoch: [49][379/380] Data 0.000 (0.003) Elapsed 2m 17s (remain 0m 0s) Loss: 0.2290(0.2005) 
EVAL: [0/191] Data 0.749 (0.749) Elapsed 0m 0s (remain 2m 45s) Loss: 0.4759(0.4759) 
EVAL: [190/191] Data 0.000 (0.016) Elapsed 0m 21s (remain 0m 0s) Loss: 1.4314(0.5196) 


Epoch 49 - avg_train_loss: 0.2005  avg_val_loss: 0.5196  time: 160s
Epoch 49 - Accuracy: 0.84670392898241


Epoch: [50][0/380] Data 0.833 (0.833) Elapsed 0m 1s (remain 9m 34s) Loss: 0.1425(0.1425) 
Epoch: [50][379/380] Data 0.000 (0.003) Elapsed 2m 17s (remain 0m 0s) Loss: 0.3150(0.2094) 
EVAL: [0/191] Data 0.619 (0.619) Elapsed 0m 0s (remain 2m 30s) Loss: 0.3948(0.3948) 
EVAL: [190/191] Data 0.000 (0.015) Elapsed 0m 21s (remain 0m 0s) Loss: 1.9160(0.5250) 


Epoch 50 - avg_train_loss: 0.2094  avg_val_loss: 0.5250  time: 160s
Epoch 50 - Accuracy: 0.8497451915173434
========== fold: 2 result ==========


In [29]:
# preds = train_loop(train, fold=Config.fold, class_num=Config.class_num, norms=Config.norms)

# 推論

## TTA

In [30]:
!pip install ttach
import ttach as tta

In [31]:
def inference(test, class_num=Config.class_num, n=0, norms=(0.5, 0.5)):
    model = BaseModel(Config.MODEL_NAME, class_num, pretrained=False)

    model.load_state_dict(torch.load(f'{Config.model_dir}/{Config.MODEL_NAME}_fold{n}_best_{Config.Version}.pth')) # 重みの読み込み
    model.to(Config.device)
    
    test_dataset = TestDataset(test, transform=get_transforms(data='valid', size=Config.size, norms=norms))
    test_loader = DataLoader(test_dataset, batch_size=Config.batch_size, shuffle=False, num_workers=Config.num_workers)
    
    all_predicts = []
    model.eval() # 推論モード
    transforms = tta.Compose(
        [
            tta.HorizontalFlip(),
            tta.VerticalFlip()
                    

        ]
    )

    tta_model = tta.ClassificationTTAWrapper(model, transforms)

    tta_model = tta.ClassificationTTAWrapper(model, transforms)

    with torch.no_grad(): # 勾配の計算をしない
        for i, (images) in tqdm(enumerate(test_loader), total=len(test_loader)):
            images = images.to(Config.device)
            y_preds = tta_model(images)
            predicts = y_preds.softmax(1).to("cpu").numpy()#.argmax(1)
            all_predicts.append(predicts)
    
    predicts = np.concatenate(all_predicts)
        
    return predicts

    # with torch.no_grad():
    # #with torch.inference_mode():
    #     for i, (images) in tqdm(enumerate(test_loader), total=len(test_loader)):
    #         images = images.to(Config.device)
    #         y_preds = model(images)
    #         predicts = y_preds.softmax(1).to("cpu").numpy()#.argmax(1)
    #         all_predicts.append(predicts)
    
    # predicts = np.concatenate(all_predicts)
        
    # return predicts

In [32]:
#MODEL_PATH = "./" # 自身の作成したモデルへのPATH

#修正
if ON_KAGGLE:
    #test = pd.read_csv(Config.data_dir + "/test.csv")
    test = pd.read_csv(Config.data_dir + "/sample_submission.csv")
    test['image_id'] = Config.data_dir + '/test_images/' + test['image_id']
else:
    #test = pd.read_csv(f'{Config.data_dir}/test/test.csv')
    test = pd.read_csv(f'{Config.data_dir}/sample_submission.csv')
    test['image_id'] = Config.data_dir + '/test_images/' + test['image_id']

In [33]:
predicts = []

for fold in range(Config.fold):
    if fold not in Config.FOLDS:
        continue

    predicts.append(inference(test, class_num=Config.class_num, n=fold))

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

In [34]:
len(predicts)

3

In [35]:
predicts

[array([[1.09295236e-07, 7.87498990e-08, 1.51694310e-10, ...,
         3.48536418e-07, 8.81978863e-08, 3.93082935e-08],
        [5.11068890e-07, 1.76540948e-06, 1.81883779e-11, ...,
         2.01636364e-11, 9.99997735e-01, 7.05232928e-10],
        [9.53523850e-04, 5.13991807e-03, 1.32104815e-05, ...,
         5.40541951e-05, 9.92766857e-01, 2.39408677e-04],
        ...,
        [9.21358279e-09, 4.37835972e-08, 4.34222613e-07, ...,
         1.13036060e-06, 9.92181484e-08, 9.86278614e-10],
        [7.64197528e-01, 2.33896628e-01, 4.37010545e-04, ...,
         9.70057663e-05, 1.12360716e-03, 2.68404383e-05],
        [1.79856556e-08, 9.67808683e-06, 1.87179396e-07, ...,
         9.99955416e-01, 8.19012484e-08, 1.73616481e-08]], dtype=float32),
 array([[5.5288768e-07, 6.4253942e-07, 1.7592194e-08, ..., 7.6102910e-06,
         8.3302535e-07, 8.1287006e-08],
        [7.9065368e-09, 6.2278727e-05, 4.5252547e-13, ..., 1.6493012e-11,
         9.9993777e-01, 6.3182112e-11],
        [2.4593081e-03

In [36]:
predicts_en = np.argmax(np.sum(predicts, axis=0), axis=1)

In [37]:
len(predicts_en)

10000

# 後処理

In [38]:
submit = pd.read_csv(Config.data_dir + "/sample_submission.csv")
submit["label"] = predicts_en # 提出用csvに書き込み
submit.to_csv('submission.csv', index=False) # csvを書き出す

In [39]:
submit['label'].value_counts()

3    1307
4    1224
0    1218
1    1134
5    1066
8    1060
7     996
9     764
6     715
2     516
Name: label, dtype: int64

In [40]:
if not ON_KAGGLE:
    submit.to_csv(f'{Config.output_dir}/{Config.Version}.csv', index=False)